In this notebook, we explore the properties of conceptnet as an evaluation benchmark

In [1]:
import pandas as pd

In [2]:
from tqdm.notebook import tqdm
labelsMap = {}
with open('../data/kgtk_conceptnet.tsv') as conceptFile:
    firstLine = True
    for line in tqdm(conceptFile):
        if firstLine:
            firstLine = False
            continue
        line = line.split('\t')
        labelsMap[line[3][1:-1]] = None
        labelsMap[line[4][1:-1]] = None

0it [00:00, ?it/s]

In [5]:
print(len(labelsMap))

1165190


In [6]:
with open('../data/labels.en.tsv') as labelsFile:
    firstLine = True
    for line in tqdm(labelsFile):
        if firstLine:
            firstLine = False
            continue
        line = line.split('\t')
        line[3] = line[3][1:-5]
        qnode, label = line[1], line[3]
#         print(qnode, label)
        if label in labelsMap:
            if labelsMap[label] == None:
                labelsMap[label] = []
            labelsMap[label].append(qnode)

0it [00:00, ?it/s]

In [7]:
conceptDF = pd.read_csv('../data/kgtk_conceptnet.tsv', sep='\t')

/nas/home/kshenoy/miniconda3/envs/kgtkEnv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [8]:
def fetchQnode(label):
    if pd.isna(label):
        return None
    label = label[1:-1]
    if label not in labelsMap:
        return None
    else:
        return labelsMap[label]

In [9]:
conceptDF['node1;qnode'] = conceptDF['node1;label'].apply(fetchQnode)
conceptDF['node2;qnode'] = conceptDF['node2;label'].apply(fetchQnode)

In [10]:
conceptDF1 = conceptDF[(~conceptDF['node1;qnode'].isna()) & (~conceptDF['node2;qnode'].isna())]

In [11]:
print(f"Concept Net dataset size reduced from {len(conceptDF)} pairs to {len(conceptDF1)} pairs")

Concept Net dataset size reduced from 3423004 pairs to 61985 pairs


In [12]:
conceptDF1['node1;qnode;len'] = conceptDF1['node1;qnode'].apply(len)
conceptDF1['node2;qnode;len'] = conceptDF1['node2;qnode'].apply(len)

<ipython-input-12-09be5c74cb96>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  conceptDF1['node1;qnode;len'] = conceptDF1['node1;qnode'].apply(len)
<ipython-input-12-09be5c74cb96>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  conceptDF1['node2;qnode;len'] = conceptDF1['node2;qnode'].apply(len)


In [13]:
conceptDF2 = conceptDF1[(conceptDF1['node1;qnode;len'] == 1) & (conceptDF1['node2;qnode;len'] == 1)]

In [14]:
print(f"Concept Net dataset size reduced from {len(conceptDF1)} pairs to {len(conceptDF2)} pairs by keeping only pairs which have one-one mapping with qnodes")

Concept Net dataset size reduced from 61985 pairs to 12395 pairs by keeping only pairs which have one-one mapping with qnodes


In [15]:
conceptDF2['relation;label'].value_counts()

related to                   6742
synonym                      2071
form of                      1240
is a                          489
etymologically related to     443
derived from                  353
manner of                     210
antonym                       185
at location                   123
similar to                    104
part of                        86
distinct from                  76
has context                    74
used for                       39
has property                   28
causes                         24
capable of                     20
entails                        13
has subevent                   11
motivated by goal               9
instance of                     8
has a                           8
made of                         8
receives action                 7
has prerequisite                5
not has property                3
causes desire                   3
has last subevent               2
genre                           2
capital       

In [20]:
conceptDF2['category'] = conceptDF2['relation;label'].apply(lambda p: 'I' if p == 'synonym' or p == 'similar to' else 'M')

<ipython-input-20-b3ad3cc600f9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  conceptDF2['category'] = conceptDF2['relation;label'].apply(lambda p: 'I' if p == 'synonym' or p == 'similar to' else 'M')


In [22]:
conceptDF2['relation;label'].apply(lambda p: 'I' if p == 'synonym' or p == 'similar to' else 'M').value_counts()

M    10220
I     2175
Name: relation;label, dtype: int64

In [18]:
conceptDF2['word1_kg_id'] = conceptDF2['node1;qnode'].apply(lambda p: p[0])
conceptDF2['word2_kg_id'] = conceptDF2['node2;qnode'].apply(lambda p: p[0])

<ipython-input-18-b837815493a6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  conceptDF2['word1_kg_id'] = conceptDF2['node1;qnode'].apply(lambda p: p[0])
<ipython-input-18-b837815493a6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  conceptDF2['word2_kg_id'] = conceptDF2['node2;qnode'].apply(lambda p: p[0])


In [21]:
conceptDF2.to_csv('../data/kgtk_conceptnet_evaluation.csv', index=None)